In [3]:
print("hello world")

hello world


In [3]:
print('here')



here


In [2]:
print("second commit")

second commit


In [1]:
print ("third commit")

third commit


In [5]:
import pandas as pd

In [8]:
pd.read_csv('Neighbourhoods.csv')

,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent
0,102011028,Avoca Beach - Copacabana,643.8000,7590,2325,738.0,46996.0,1906.0
1,102011029,Box Head - MacMasters Beach,3208.6000,10986,3847,907.0,42621.0,1682.0
2,102011030,Calga - Kulnura,76795.1000,4841,1575,1102.0,42105.0,1182.0
3,102011031,Erina - Green Point,3379.3000,14237,4450,1666.0,43481.0,1595.0
4,102011032,Gosford - Springfield,1691.2000,19385,6373,2126.0,45972.0,1382.0
...,...,...,...,...,...,...,...,...
317,106011109,Cessnock Region,1570.4341,"7,931","3,281",673.0,73164.0,1080.0
318,106011113,Singleton Region,4067.2349,"4,919","2,055",698.0,87984.0,1000.0
319,111021218,Morisset - Cooranbong,330.5208,"14,959","6,298",1154.0,58084.0,1260.0
320,114021285,Hill Top - Colo Vale,174.3752,"6,025","2,249",400.0,81120.0,1512.0
